In [84]:
from ocpmodels.models.rhescnv8clean.tests.utils import test_context

escnctx = test_context("/workspaces/ocp/configs/s2ef/2M/escn/escn-ckpt.yml").__enter__()

2024-04-06 21:23:49 (WARNING): Detected old config, converting to new format. Consider updating to avoid potential incompatibilities.


2024-04-06 21:23:49 (INFO): amp: true
cmd:
  checkpoint_dir: ./checkpoints/2024-04-06-21-24-16
  commit: 6e3cd9b
  identifier: ''
  logs_dir: ./logs/wandb/2024-04-06-21-24-16
  print_every: 10
  results_dir: ./results/2024-04-06-21-24-16
  seed: 0
  timestamp_id: 2024-04-06-21-24-16
dataset:
  format: trajectory_lmdb
  grad_target_mean: 0.0
  grad_target_std: 2.887317180633545
  key_mapping:
    force: forces
    y: energy
  normalize_labels: true
  src: /mnt/datasets/s2ef/2M/train/
  target_mean: -0.7554450631141663
  target_std: 2.887317180633545
  transforms:
    normalizer:
      energy:
        mean: -0.7554450631141663
        stdev: 2.887317180633545
      forces:
        mean: 0.0
        stdev: 2.887317180633545
eval_metrics:
  metrics:
    energy:
    - mae
    forces:
    - forcesx_mae
    - forcesy_mae
    - forcesz_mae
    - mae
    - cosine_similarity
    - magnitude_error
    misc:
    - energy_forces_within_threshold
gpus: 1
logger: wandb
loss_fns:
- energy:
    coeffic

In [89]:
import lovely_tensors as lt

lt.monkey_patch()

In [90]:
mapping = escnctx.trainer.model.coeff_mapping()

print(mapping.m_size)
print(mapping.to_m)

mmax = mapping.mmax_list[0]
print(mmax)

[5 4 3]
tensor[19, 19] n=361 (1.4Kb) x∈[0., 1.000] μ=0.053 σ=0.224 cuda:0
2


In [92]:
from bidict import bidict

i = 0
l_m_mapping = bidict[tuple[int, int], int]()
for l in range(escnctx.trainer.model.lmax_list[0] + 1):
    for m in range(-l, l + 1):
        if abs(m) > mmax:
            continue
        l_m_mapping[(l, m)] = i
        i += 1

l_m_mapping

bidict({(0, 0): 0, (1, -1): 1, (1, 0): 2, (1, 1): 3, (2, -2): 4, (2, -1): 5, (2, 0): 6, (2, 1): 7, (2, 2): 8, (3, -2): 9, (3, -1): 10, (3, 0): 11, (3, 1): 12, (3, 2): 13, (4, -2): 14, (4, -1): 15, (4, 0): 16, (4, 1): 17, (4, 2): 18})

In [93]:
min_m = min(l_m_mapping.keys(), key=lambda x: x[1])[1]
max_m = max(l_m_mapping.keys(), key=lambda x: x[1])[1]
min_m, max_m

(-2, 2)

In [101]:
mapping = escnctx.trainer.model.coeff_mapping()

new_vector = [None] * mapping.to_m.shape[0]

i = 0
for m_abs in range(0, max_m + 1):
    m_text = []
    count = 0
    for m in [m_abs, -m_abs] if m_abs > 0 else [0]:
        for l in range(m_abs, escnctx.trainer.model.lmax_list[0] + 1):
            old = l_m_mapping[(l, m)]

            mapping_vector = mapping.to_m[:, old]
            # Make sure all are 0 except one, which is 1
            assert mapping_vector.count_nonzero() == 1
            assert mapping_vector.sum() == 1

            # Get the integer index of the 1
            new = mapping_vector.argmax()
            # print(f"l={l}, m={m}, [old={old}] => [new={new}]")
            m_text.append(f"l={l}, m={m}, [old={old}] => [new={new}]")
            count += 1

            new_vector[old] = i

        # print()
        m_text.append("")

    print(f"|m|={m_abs}; count={count}")
    for t in m_text:
        print(t)

    print()

|m|=0; count=5
l=0, m=0, [old=0] => [new=0]
l=1, m=0, [old=2] => [new=1]
l=2, m=0, [old=6] => [new=2]
l=3, m=0, [old=11] => [new=3]
l=4, m=0, [old=16] => [new=4]


|m|=1; count=8
l=1, m=1, [old=3] => [new=5]
l=2, m=1, [old=7] => [new=6]
l=3, m=1, [old=12] => [new=7]
l=4, m=1, [old=17] => [new=8]

l=1, m=-1, [old=1] => [new=9]
l=2, m=-1, [old=5] => [new=10]
l=3, m=-1, [old=10] => [new=11]
l=4, m=-1, [old=15] => [new=12]


|m|=2; count=6
l=2, m=2, [old=8] => [new=13]
l=3, m=2, [old=13] => [new=14]
l=4, m=2, [old=18] => [new=15]

l=2, m=-2, [old=4] => [new=16]
l=3, m=-2, [old=9] => [new=17]
l=4, m=-2, [old=14] => [new=18]




In [95]:
for i in range(len(new_vector)):
    if new_vector[i] is not None:
        continue

    l, m = l_m_mapping.inverse[i]
    print(f"l={l}, m={m} is missing")